# Monada Reader

- ### Pozwala na opóźnienie podania wybranych parametrów do momentu uruchomienia
- ### Służy do wstrzykiwania zależności do czystych funkcji

## Podstawowy przykład

In [ ]:
from datetime import datetime
from reader import Reader

# Operacja w obszarze logiki biznesowej aplikacji
# na tym etapie nie mamy (albo nie chcemy mieć)
# dostępu do zależności których ta funkcja potrzebuje
@Reader.create
def do_something_basic(value: int) -> int:
    multiplier = yield lambda env: env["multiplier"]
    logger = yield lambda env: env["logger"]
    output = value * multiplier
    logger(f"Wynik: {output}")
    return output

# "Wywołanie" funkcji, z pominięciem zależności.
# W rzeczywistości uruchomi się później, dopiero gdy dostarczymy zależności
output_reader = do_something_basic(5)

# =======================================================
# Gdzieś w górnej warstwie aplikacji, na etapie montowania aplikacji
# gdzie mamy dostęp do wszystkich potrzebnych zależności

env = {
    "multiplier": 10,
    "logger": lambda msg: print(datetime.now().isoformat(), "|", msg)
}

# Wstrzyknięcie zależności w funkcję, co powoduje jej uruchomienie
output_reader.run(env)

## Kompozycja i potoki danych

In [ ]:
from utils import curry

@Reader.create
@curry
def do_something_complex(base_value: int, divisor: int) -> str:
    value = yield do_something_basic(base_value)
    final_value = value / divisor
    return f"Ostateczna wartość: {final_value}"

@Reader.create
def log_and_forward(value):
    logger = yield lambda env: env["logger"]
    logger(f"Zalogowana wartość: {value}")
    return value

# Zbudowanie potoku przepuszczającego wynik pierwszej funkcji
# przez dwie następne funkcje, na wzór potoku w powłoce systemów *nix
complex_output_reader = (
    do_something_complex(6, 10)
    | (lambda text: text.upper())
    | log_and_forward
)

# Uruchomienie potoku przez podanie zależności
complex_output_reader.run(env)


In [ ]:
# Alternatywna implementacja wykorzystając kompozycję podstawowych Reader-ów,
# przed podaniem bardziej złożonego Reader-a do potoku
upper_log_and_forward = (
    Reader.create(lambda text: "PIPE[" + text.upper() + "]")
    >> log_and_forward
)

complex_output_reader = (
    do_something_complex(6, 10)
    | upper_log_and_forward
)

complex_output_reader.run(env)